In [2]:
import pandas as pd
from datasets import DatasetDict, Dataset, load_from_disk
import os

#os.environ['WANDB_DISABLED'] = 'true'

dataset_path = '../bin/multirc_dataset.hf'
seed = 42

df = pd.read_csv('../data/multirc.csv')#[['source_text', 'source_text_clean', 'question_text', 'answer_text', 'is_correct']]
def clean_df(df):
    df['labels'] = df['is_correct'].apply(lambda x: str(x))
    df['answer_text'] = df['answer_text'].apply(lambda x: str(x))
    df['comp'] = "### Instructions \nYou will be provided with a text, a question, and an answer. Your task is to decide whether the answer is correct or incorrect and respond with a single word denoting the appropriate label: `False` for incorrect and `True` for correct.\n\n\n###The text\nThe text is as follows, delimited by three backticks: ```" + df['source_text_clean'] + "...```\n\n\n ### The question and answer pair \n The question and answer pair are as follows, delimited by three backticks: ```##Question: " + df["question_text"] + " ##Answer: " + df['answer_text'] + "``` "
    df['text'] = df['comp'] + "\n\n\n<<<" + df['labels'] + ">>>"
    df = df[['comp', 'labels', 'text']]

     
    return df.reset_index()
    return df
    
from sklearn.model_selection import train_test_split
source_index = df['source_text'].drop_duplicates()
train_index, validtest_index = train_test_split(source_index, test_size=0.3,train_size=0.7, random_state = seed)
valid_index, test_index = train_test_split(validtest_index, test_size=0.5, train_size=0.5, random_state = seed)

train_df = clean_df(df[df['source_text'].isin(train_index)].dropna())[['comp', 'labels', 'text']]
valid_df = clean_df(df[df['source_text'].isin(valid_index)].dropna())[['comp', 'labels', 'text']]
test_df = clean_df(df[df['source_text'].isin(test_index)].dropna())[['comp', 'labels', 'text']]

print(len(train_df))
print(len(valid_df))
print(len(test_df))

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df, preserve_index=False)
ds['valid'] = Dataset.from_pandas(valid_df, preserve_index=False)
ds['test'] = Dataset.from_pandas(test_df, preserve_index=False)

19170
4080
3962


In [3]:
from peft import AutoPeftModelForCausalLM
import torch
from transformers import AutoTokenizer
device_map = {"": 0}
model_m = AutoPeftModelForCausalLM.from_pretrained("multirc-llama", device_map=device_map, torch_dtype=torch.bfloat16, do_sample=True)
base_model_name = "ehartford/dolphin-llama2-7b"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, padding=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import torch
import gc
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [1]:
from time import perf_counter

results = []
times = []
counter = 0
for text in ds['test']['comp']:
    start_time = perf_counter()
    print('Processing item ' +str(counter)+ ' of ' + str(len(ds['test']['comp'])), end = '\r')
    inputs = tokenizer(text, return_tensors='pt').to(device)
    outputs = model_m.generate(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'], max_new_tokens=1700, pad_token_id=tokenizer.eos_token_id, max_length=2046)
    res = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append(res)
    times.append(perf_counter()-start_time)
    counter += 1

NameError: name 'ds' is not defined

In [ ]:
df = ds['test'].to_pandas()
df['results'] = results
df['times'] = times
df.to_csv('llama-results.csv')